In [1]:
# install dependencies
! pip install -U pymoo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 8.2 MB/s 
     |████████████████████████████████| 249 kB 46.8 MB/s 
     |████████████████████████████████| 80 kB 4.2 MB/s 
     |████████████████████████████████| 207 kB 39.3 MB/s 
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210096 sha256=1669db2f9214e7ec30a1274fc900325326bc6010d5872079c4d7d3f13577a9e5
  Stored in directory: /root/.cache/pip/wheels/2d/08/6b/126ea9009f7482fd53a78d0db2ece5aca70af8f4a30445386b
Successfully built grapheme


# 1 the Probability of weights 

In [2]:
from numpy.core.memmap import uint8
import numpy as np

class AdaptiveWeight:
    def __init__(self, N, Max_iter, F, K):
        self.Ns = np.zeros(( Max_iter+1, N))
        self.As = np.zeros(( Max_iter+1, N))
        self.Is = np.zeros(( Max_iter+1, N))
        self.Iter = 0
        self.Max_iter = Max_iter
        self.N_track = N
        self.pobWeights = np.ones((N,))/N
        self.weights = np.ones((N,))
        self.Ns[0] = np.ones((N,))
        self.As[0] = np.ones((N,))
        self.Is[0] = np.ones((N,))
        self.F = F
        self.K = K
        self.listIndexs = list(np.arange(N, dtype=np.uint8))

    def random_tracks(self, size):
        tracksIds = np.random.choice(self.listIndexs, size, p=self.pobWeights)
        return tracksIds

    def update(self, tracks, objects, bestI):
        self.Iter += 1
        self.Is[self.Iter, bestI] = 1
        for i in range(self.N_track):
            indexs = np.where(tracks==i)[0]
            self.Ns[self.Iter, i] = len(indexs)
            self.As[self.Iter, i] = np.mean(objects[indexs])
            if len(indexs) == 0:
                self.As[self.Iter, i] = 0
            Ii = np.sum(self.Is[:self.Iter, i])

            new_weight = (self.F*self.Ns[self.Iter, i] + 
                              (1-self.F)*self.As[self.Iter, i] + self.K*Ii)
            self.weights[i] =  max(0.3*self.weights[i] + 0.7*new_weight, 0)
        total = np.sum(self.weights)
        #print(self.weights, total)
        for i in range(self.N_track):
            self.pobWeights[i] = self.weights[i]/total



# 2 AMIS class

In [27]:
class AMIS:
    def __init__(self, problem, pop_size,  max_iter, CR=0.3, CRT=0.2, F1=0.6, F2=0.4, lo=0.5):
        self.pop_size = pop_size
        self.CR = CR
        self.problem = problem
        self.max_iter = max_iter
        self.Xs = None
        self.fitnessXs = None
        self.bestX = None
        self.best2X = None
        self.CRT = CRT
        self.bestFitness = 0
        self.F1 = F1
        self.F2 = F2
        self.lo = lo
        self.data = np.zeros((3*max_iter, self.problem.n_var))
        self.out = {}
        self.initialize()
        self.N_blackboxs = 8
        self.adaptiveWeight = AdaptiveWeight(self.N_blackboxs, max_iter, 0.05, 1)

    def initialize(self):
        self.Xs = np.random.rand( self.pop_size, self.problem.n_var)
        self.fitnessXs = np.zeros((self.pop_size,))
        self.cal_fitness()
        max_index = np.argmax(self.fitnessXs)
        self.bestX = np.copy(self.Xs[max_index])
        self.best2X = np.copy(self.bestX)
        self.bestFitness = self.fitnessXs[max_index]
        
    def cal_fitness(self):
        for i in range(self.pop_size):
            self.problem._evaluate(self.Xs[i], self.out)
            self.fitnessXs[i] = self.out['F']
        
    def IB1_best(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.lo*self.Xs[index_r1s[:N]] + self.F1*(self.bestX - self.Xs[index_r1s[:N]]) +\
                                              self.F2*(self.Xs[index_r2s[:N]] - self.Xs[index_r3s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs


    def IB2_best2(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.Xs[index_r1s[:N]] + self.F1*(self.bestX - self.Xs[index_r1s[:N]]) +\
                                              self.F2*(self.best2X - self.Xs[index_r1s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs

    def IB3_DE(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.Xs[index_r1s[:N]] + self.F1*(self.best2X - self.Xs[index_r1s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs

    def IB4_DER(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.Xs[index_r1s[:N]] + np.random.rand()*(self.best2X - self.Xs[index_r1s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs

    def IB5_random(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs = rs
        return Vs
    
    
    def IB6_random_transit(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs = (1-cross_select)*xs + cross_select*rs
        return Vs
    
    def IB7_best_transit(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs = (1-cross_select)*xs + cross_select*self.bestX
        return Vs

    def IB8_scale(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs =  ((1 - cross_select)*rs)*xs +   cross_select*xs
        return Vs

    def select_blocks(self):
        self.Vs = np.copy(self.Xs)
        for i in range(self.N_blackboxs):
            indexI = list(np.where(self.currentTracks == i)[0])
            if len(indexI) == 0:
                continue
            if i == 0:
                self.Vs[indexI] = self.IB1_best(self.Xs[indexI])
            elif i==1:
                self.Vs[indexI] = self.IB2_best2(self.Xs[indexI])
            elif i==2:
                self.Vs[indexI] = self.IB3_DE(self.Xs[indexI])
            elif i==3:
                self.Vs[indexI] = self.IB4_DER(self.Xs[indexI])
            elif i==4:
                self.Vs[indexI] = self.IB5_random(self.Xs[indexI])
            elif i == 5:
                self.Vs[indexI] = self.IB6_random_transit(self.Xs[indexI])
            elif i==6:
                self.Vs[indexI] = self.IB7_best_transit(self.Xs[indexI])
            elif i==7:
                self.Vs[indexI] = self.IB8_scale(self.Xs[indexI])
            
            

    def recombination(self):
        cross_select = np.random.choice(a=[0, 1], size=self.Xs.shape, p=[self.CR, 1-self.CR])  
        self.Us = (1-cross_select)*self.Xs + cross_select*self.Vs
        
    def accept_all(self):
        fitnessUs = np.zeros((self.pop_size,))
        select_mask = np.zeros((self.pop_size,))
        for i in range(self.pop_size):
            self.problem._evaluate(self.Us[i], self.out)
            fitnessUs[i] = self.out['F']
            self.fitnessXs[i] = fitnessUs[i]
            self.Xs[i] = np.copy(self.Us[i])
            if fitnessUs[i] > self.bestFitness:
                self.bestFitness = fitnessUs[i]
                self.best2X = np.copy(self.bestX)
                self.bestX = np.copy(self.Us[i])

    def selection(self):
        fitnessUs = np.zeros((self.pop_size,))
        select_mask = np.zeros((self.pop_size,))
        for i in range(self.pop_size):
            self.problem._evaluate(self.Us[i], self.out)
            fitnessUs[i] = self.out['F']
            delta = fitnessUs[i] - self.fitnessXs[i]
            if (delta < 0 or 
                np.random.rand() + 5 < np.exp(-delta*100) or
                np.random.rand()  < 0.00001):
                self.fitnessXs[i] = fitnessUs[i]
                self.Xs[i] = np.copy(self.Us[i])
                if fitnessUs[i] < self.bestFitness:
                    self.bestFitness = fitnessUs[i]
                    self.best2X = np.copy(self.bestX)
                    self.bestX = np.copy(self.Us[i])



    def iterate(self):
        for it in range(self.max_iter):
            self.currentTracks = self.adaptiveWeight.random_tracks(self.pop_size)
            self.select_blocks()
            arg_max = np.argmax(self.fitnessXs)
            self.adaptiveWeight.update(self.currentTracks, self.fitnessXs, self.currentTracks[arg_max])
            self.recombination()
            self.selection()

            idx= np.argsort(self.bestX)

            print(it, round(np.mean(self.fitnessXs), 4), round(self.bestFitness, 4), self.adaptiveWeight.pobWeights)
            arg_maxs = np.argsort(self.fitnessXs)
            self.data[it*3:3*(it+1)] = np.copy(self.Xs[arg_maxs[:3]])

In [28]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import math

class SampleProblem(ElementwiseProblem):
    
    def __init__(self, N):
        self.N = N
        super().__init__(n_var=self.N, n_obj=1, n_constr=0, xl=0, xu=1)

    def _function_cost(self, x):
        x = x[0]
        return x*x - 0.5

    def _evaluate(self, x, out, *args, **kwargs):
        
        f = self._function_cost(x)
        

        out["hash"] = hash(str(x))
        out["F"] = f
        #out["pheno"] = {"values": solution}

In [29]:
from pymoo.core.callback import Callback
from pymoo.optimize import minimize


problem = SampleProblem(1)
np.random.seed(0)

algorithm = AMIS(problem,
    pop_size=20,
    CR=0.3,
    max_iter = 1000,
    #dither="vector",
    #jitter=False
)
algorithm.iterate()

0 -0.2252 -0.5 [0.09778194 0.10157395 0.13701008 0.117158   0.14360524 0.1380563
 0.12407945 0.14073503]
1 -0.3212 -0.5 [0.11775368 0.11463643 0.09394629 0.09473614 0.21044061 0.12867541
 0.10993892 0.12987251]
2 -0.3768 -0.5 [0.11581941 0.09960191 0.10109663 0.08115009 0.20597094 0.19188819
 0.11103982 0.09343302]
3 -0.4221 -0.5 [0.18329167 0.07837193 0.07974707 0.0871531  0.19880932 0.19941474
 0.08941611 0.08379606]
4 -0.4802 -0.5 [0.16638678 0.08761459 0.06901982 0.09507904 0.18367537 0.25622998
 0.06985699 0.07213742]
5 -0.4844 -0.5 [0.15984739 0.15317788 0.06037208 0.06691172 0.15556008 0.26346916
 0.05629761 0.08436408]
6 -0.4952 -0.5 [0.21876194 0.15694635 0.05224691 0.05725844 0.1403283  0.23507295
 0.07704586 0.06233924]
7 -0.4961 -0.5 [0.21595597 0.19845559 0.05036518 0.04859117 0.12884871 0.22870157
 0.05498673 0.07409507]
8 -0.4966 -0.5 [0.27003157 0.19102251 0.04724905 0.0662393  0.12154267 0.20470346
 0.04562555 0.05358589]
9 -0.4966 -0.5 [0.32000295 0.18647508 0.0423927

## 3 Avg Ensemble 

In [31]:
# The setup

import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from skimage import transform, io
from os import listdir

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import Model, models
from keras import backend as k
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, auc, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt


In [35]:
def avg_ensemble(model_name, preds, ys):
  ypred1, ypred2, ypred3, ypred4, ypred5 = preds
  print('Report :',datetime.datetime.now())
  print('\n')
  ypred_max_m1 = np.argmax(ypred1, axis=1)
  print('Model 1 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m1))
  print('Model 1 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred1, multi_class='ovr'))
  print('Model 1 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m1,average='weighted'))
  print('\n')
  ypred_max_m2 = np.argmax(ypred2, axis=1)
  print('Model 2 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m2))
  print('Model 2 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred2, multi_class='ovr'))
  print('Model 2 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m2,average='weighted'))
  print('\n')
  ypred_max_m3 = np.argmax(ypred3, axis=1)
  print('Model 3 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m3))
  print('Model 3 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred3, multi_class='ovr'))
  print('Model 3 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m3,average='weighted'))
  print('\n')
  ypred_max_m4 = np.argmax(ypred4, axis=1)
  print('Model 4 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m4))
  print('Model 4 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred4, multi_class='ovr'))
  print('Model 4 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m4,average='weighted'))
  print('\n')
  ypred_max_m5 = np.argmax(ypred5, axis=1)
  print('Model 5 : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_m5))
  print('Model 5 : ', model_name, ' has AUC ',roc_auc_score(ys, ypred5, multi_class='ovr'))
  print('Model 5 : ', model_name, ' has F1 ',f1_score(ys, ypred_max_m5,average='weighted'))
  print('\n')
  
  ypred_en = (ypred1 + ypred2 + ypred3 + ypred4 + ypred5)/5
  ypred_max_en = np.argmax(ypred_en, axis=1)
  print('Ensemble Model : ', model_name, ' has Accuracy ',accuracy_score(ys, ypred_max_en))
  print('Ensemble Model : ', model_name, ' has AUC ',roc_auc_score(ys, ypred_en, multi_class='ovr'))
  print('Ensemble Model : ', model_name, ' has F1 ',f1_score(ys, ypred_max_en,average='weighted'))

In [ ]:

from keras import models
from keras import backend as k

if k.backend() == 'tensorflow':
  print('Clear old session')
  k.clear_session()

model_path = "./Model"
model_name = "MobileNetV2"

Xs = np.load("./Dataset/Xs.npy")
ys = np.load("./Dataset/Xs.npy")

print('Report :',datetime.datetime.now())
print('\n Load Model ', model_name)
model1 = models.load_model(model_path+model_name+"_1.tf")
model2 = models.load_model(model_path+model_name+"_2.tf")
model3 = models.load_model(model_path+model_name+"_3.tf")
model4 = models.load_model(model_path+model_name+"_4.tf")
model5 = models.load_model(model_path+model_name+"_5.tf")

print('\n Predict')
start = datetime.datetime.now()
ypred1 = model1.predict(Xs*255)
ypred2 = model2.predict(Xs*255)
ypred3 = model3.predict(Xs*255)
ypred4 = model4.predict(Xs*255)
ypred5 = model5.predict(Xs*255)

ypreds = [ypred1, ypred2, ypred3, ypred4,ypred5]


In [ ]:
avg_ensemble(model_name, ypreds, ys)

# 4 AMIS-WCE

In [38]:
from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
from pymoo.algorithms.soo.nonconvex.de import DE
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import math

class WC_EnsembleProblem(ElementwiseProblem):
    
    def __init__(self,preds, ys_c):
        self.N = len(preds)
        self.preds = preds
        self.ys_c = ys_c
        super().__init__(n_var=self.N, n_obj=1, n_constr=0, xl=0, xu=1)

    

    def _evaluate(self, xs, out, *args, **kwargs):
        w = (xs )/np.sum(xs)
        ypred_en = self.preds[0]*w[0]
        for i in range(1, len( self.preds)):
            ypred_en += (self.preds[i]*w[i])

        indexs = np.where(ypred_en>1)
        ypred_en[indexs] = 1

        ypred_max_en = np.argmax(ypred_en, axis=1)
        d = ypred_en - self.ys_c
      

        f = 1 - accuracy_score(ys, ypred_max_en)
        
        out["hash"] = hash(str(w))
        out["F"] = f
        out["pheno"] = {"weights": w}

In [ ]:
from pymoo.core.callback import Callback
from pymoo.optimize import minimize
from tensorflow.keras.utils import to_categorical

y_c = to_categorical(ys, 5)
problem = WC_EnsembleProblem(ypreds, y_c)
np.random.seed(0)

algorithm = AMIS(problem,
    pop_size=20,
    CR=0.3,
    max_iter = 1000,
    #dither="vector",
    #jitter=False
)
algorithm.iterate()

# 5 AMIS-WCLE

In [40]:
from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
from pymoo.algorithms.soo.nonconvex.de import DE
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import math

class WLC_EnsembleProblem(ElementwiseProblem):
    
    def __init__(self,preds, ys_c, num_clases):
        self.N = len(preds)
        self.preds = preds
        self.ys_c = ys_c
        super().__init__(n_var=self.N, n_obj=1, n_constr=0, xl=0, xu=1)
    

    def _evaluate(self, xs, out, *args, **kwargs):
        w = (xs )/np.sum(xs)
        ypred_en = self.preds[0]*w[0]
        for i in range(1, len( self.preds)):
            ypred_en += (self.preds[i]*w[i])

        indexs = np.where(ypred_en>1)
        ypred_en[indexs] = 1

        ypred_max_en = np.argmax(ypred_en, axis=1)
        d = ypred_en - self.ys_c
      

        f = 1 - accuracy_score(ys, ypred_max_en)
        
        out["hash"] = hash(str(w))
        out["F"] = f
        out["pheno"] = {"weights": w}

In [41]:
y_c = to_categorical(ys, 5)
problem = WLC_EnsembleProblem(ypreds, y_c, 5)
np.random.seed(0)

algorithm = AMIS(problem,
    pop_size=20,
    CR=0.3,
    max_iter = 1000,
    #dither="vector",
    #jitter=False
)
algorithm.iterate()